In [2]:
import pickle
import math

city = "SF"

event_ids = pickle.load( open( city+"/event_ids.pickle", "rb" ) )
master_venue_dict = pickle.load( open( city+"/regional_venue_set.pickle", "rb" ) )
regional_good_venues = pickle.load( open( city+"/regional_good_venues.pickle", "rb" ) )
master_genre_map = pickle.load( open( city+"/master_genre_map.pickle", "rb" ) )
venue_ids = pickle.load( open( city+"/venue_ids.pickle", "rb" ) )
venue_list = pickle.load( open( city+"/venue_list.pickle", "rb" ) )
region_artists = pickle.load( open( city+"/regional_subset.pickle", "rb" ) )
locations = pickle.load( open( city+"/locations.pickle", "rb" ) )

#print event_ids

from sklearn import cluster, datasets
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt

def getIndex(element, my_list):
    # Create a generator
    gen = (i for i,x in enumerate(my_list) if x == element)
    for i in gen:
        return i

# Let's prepare the master_venue_vect:
master_venue_vect = []
master_venue_list = []
master_venue_locs = []

for key, val in regional_good_venues.iteritems():
    master_venue_vect.append(val)
    master_venue_list.append(key)

## Let's normalize all genre vectors for venues
for i in range(len(master_venue_vect)):
    master_venue_vect[i] = master_venue_vect[i]/np.linalg.norm(master_venue_vect[i])

pickle.dump( master_venue_vect, open( city+"/master_venue_vect.pickle", "wb" ) )
pickle.dump( master_venue_list, open( city+"/master_venue_list.pickle", "wb" ) )

X_ven = np.array(master_venue_vect)
#y_ven = np.array(venue_ids)
all_of_them = []
lng_of_them = []
std_of_them = []

k_rng = range(30,31)
#k_rng = range(3,60)

for k in k_rng:
    k_means = cluster.KMeans(n_clusters=k,  init='k-means++', max_iter=250, n_init=20,)
    #k_means = cluster.AgglomerativeClustering(n_clusters=k, affinity='cosine', linkage='complete')
    #k_means = cluster.AffinityPropagation(affinity='euclidean')
    k_means.fit(X_ven) 
    pickle.dump( k_means, open( city+"/k_means.pickle", "wb" ) )

    # Index = cluster number, append to list of venues
    clustered_venue_names = []
    clustered_venue_ids = []
    clustered_venue_locations = []
    #print k_means.labels_
    # For loop over clusters
    for cluster_num in range(k):
        temp1 = [] # names
        temp2 = [] # ids
        temp3 = [] # lat/lons
        for i in range(len(master_venue_vect)):
            if k_means.labels_[i] == cluster_num:
                #print "Here in cluster number:", cluster_num, i, venue_list[i]
                temp1.append(master_venue_list[i])
                temp2.append(venue_ids[i])
                temp3.append(locations[i])
        clustered_venue_names.append(temp1)
        clustered_venue_ids.append(temp2)
        clustered_venue_locations.append(temp3)
    #print clustered_venues
    all_sil_samps = np.array(metrics.silhouette_samples(X_ven, k_means.labels_))
    all_of_them.append(all_sil_samps.mean())
    
    # Let's compute some metrics
    hold_x_vals = []
    for i in range(len(clustered_venue_names)):
        hold_x_vals.append(len(clustered_venue_names[i]))
    # append average venue number, standard dev
    lng_of_them.append( np.mean(np.array(hold_x_vals)) )
    std_of_them.append( np.std(np.array(hold_x_vals)) )


#print all_of_them
for i in range(len(clustered_venue_names)):
    print "Cluster:", i, "length", 
    for ddd in clustered_venue_names[i]:
        print ddd, # clustered_venue_locations[i]
    print "\n"



x_pos = np.array(k_rng)
plt.xlim([min(x_pos)-1, max(x_pos)+1])
'''
# Silhouette
data = np.array(all_of_them)
for d, x in zip(data, x_pos):
    plt.errorbar(x=x, y=d, fmt='o', color='k')

plt.ylim([0,.5])
plt.ylabel('Silhouette score')
plt.title('Silhouette vs. k')

'''

# Mean venues per cluster
data = np.array(lng_of_them)
std = np.array(std_of_them)
for d, x, errs in zip(data, x_pos, std):
    plt.errorbar(x=x, y=d, yerr=errs, fmt='o', color='k')

plt.ylim([0,50])
plt.ylabel('Mean venues')
plt.title('Venues per cluster')


plt.show()
'''
hold_vecs = []
for key, val in master_venue_dict.iteritems():
    hold_vecs.append(np.sqrt(val.dot(val)))

hold_vecs.sort(reverse=True)

plt.bar(range(len(hold_vecs)), hold_vecs )

plt.title("Genre Presence")
plt.xlabel("Venue")
#plt.yscale('log', nonposy='clip')
plt.ylabel("Net frequency")
plt.show()
'''
#pickle.dump( k_means, open( "k_means.pickle", "wb" ) )

Cluster: 0 length Swedish American Hall Amnesia Doc's Lab The Grand Ballroom @ Regency Center The Lost Church Hopmonk Novato Beer Garden Don Quixote's International Music Hall Freight & Salvage Coffee House Viracocha The Crepe Place 

Cluster: 1 length The Fillmore Hotel Utah Saloon The Warfield Yerba Buena Gardens The Night Light Leo's Potrero Del Sol Park Jack London Square Subrosa One Fam Community Center The Independent Caffe Pergolesi Golden Gate Park Milk Bar El Rio California Academy of Sciences Greek Theatre The Octopus Literary Salon Slim's Rickshaw Stop Great American Music Hall The Chapel Bottom of the Hill Bimbo's 365 Club Palace of Fine Arts Fox Theater Geary Temple aka Fillmore Neck of the Woods 

Cluster: 2 length 1-2-3-4 Go! Records AT&T Park 50 Mason Social House Longboard Margarita Bar SAP Center Wente Vineyards Flint Center for the Performing Arts Dolores Park Cafe Red Devil Lounge City National Civic Event Center, SJSU The Masonic Mountain Winery The Starry Plough J

/Users/ericwhite/SceneHash/venv/lib/python2.7/site-packages/numpy/core/_methods.py:59: RuntimeWarning: Mean of empty slice.
  warnings.warn("Mean of empty slice.", RuntimeWarning)
/Users/ericwhite/SceneHash/venv/lib/python2.7/site-packages/numpy/core/_methods.py:71: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


'\nhold_vecs = []\nfor key, val in master_venue_dict.iteritems():\n    hold_vecs.append(np.sqrt(val.dot(val)))\n\nhold_vecs.sort(reverse=True)\n\nplt.bar(range(len(hold_vecs)), hold_vecs )\n\nplt.title("Genre Presence")\nplt.xlabel("Venue")\n#plt.yscale(\'log\', nonposy=\'clip\')\nplt.ylabel("Net frequency")\nplt.show()\n'

In [7]:
import requests
import math
import pickle

token = 'WL4JBSOI4ZGXPY8FN'
artist = "weezer"
query_params = { 'api_key': token,
                 'name': artist,
                 'bucket': 'familiarity',
                 'bucket': 'terms',
         }

endpoint = 'http://developer.echonest.com/api/v4/artist/profile?'
response = requests.get( endpoint, params=query_params)
full_data = response.json()

if response.json()['response']['status']['message'] != 'Success'

terms = full_data['response']['artist']['terms']

genre_map = pickle.load( open( "genre_map.pickle", "rb" ) )
k_means = pickle.load( open( "k_means.pickle", "rb" ) )
clustered_venue_locations = pickle.load( open( "clustered_venue_locations.pickle", "rb" ) )
clustered_venue_names = pickle.load( open( "clustered_venue_names.pickle", "rb" ) )
# Now don't touch genre_map!
def getIndex(genre, mapped):
    gen = (i for i,x in enumerate(mapped) if x == genre)
    for i in gen:
        return i

genre_vect = [0] * len(genre_map)
    
for term in terms:
    index = getIndex(term['name'], genre_map)
    if index is None:
        continue
    genre_vect[index] = term['frequency']

# Normalize
sum_sq = 0
for component in genre_vect:
    sum_sq += component*component
denom = math.sqrt(sum_sq)
# Now go back through and divide each by denom
for j in range(len(genre_vect)):
    if denom > 0:
        genre_vect[j] = genre_vect[j]/denom
    else:
        genre_vect[j] = 0.

p_index = k_means.predict(genre_vect)

full_locations = []
best_venues = []
for i in range(len(clustered_venue_names[p_index[0]])):
    full_locations.append(float(clustered_venue_locations[p_index[0]][i][0]))
    full_locations.append(float(clustered_venue_locations[p_index[0]][i][1]))
    best_venues.append(clustered_venue_names[p_index[0]][i])
print full_locations
print best_venues

SyntaxError: invalid syntax (<ipython-input-7-3966b85e84c3>, line 17)

In [8]:
from sklearn.decomposition import PCA

pca = PCA()

sk_tran = pca.fit_transform(X_ven)

print(pca.explained_variance_)
print(type(pca.explained_variance_))

xx_pos = np.array(range(len(X_ven)))
plt.xlim([min(xx_pos)-1, max(xx_pos)+1])

# Silhouette
pca_data = pca.explained_variance_
for d, x in zip(pca_data, xx_pos):
    plt.errorbar(x=x, y=d, fmt='o', color='k')

plt.ylim([0,.08])
plt.ylabel('Variance')
plt.title('Explained Variance, Venues (PCA)')
plt.xlabel('Venue')
plt.show()

#plt.figure(1, figsize=(6, 4))
#plt.clf()
#plt.axes([.2, .2, .7, .7])
#plt.plot(np.log(pca.explained_variance_), linewidth=2)
#plt.axis('tight')
#plt.xlabel('Components')
#plt.ylabel('log(explained var.)')

#show(mpl.to_bokeh(name='plt'))

[  6.61393504e-02   6.31718429e-02   4.48657947e-02   3.61407688e-02
   3.48508424e-02   3.30501275e-02   2.62915009e-02   2.35773310e-02
   2.14418549e-02   2.02148597e-02   1.88683796e-02   1.83818671e-02
   1.65609411e-02   1.47180639e-02   1.45998207e-02   1.42085474e-02
   1.36464405e-02   1.20796189e-02   1.17234274e-02   1.05132593e-02
   9.75039112e-03   9.61377587e-03   9.07907975e-03   8.80273857e-03
   8.36241622e-03   8.02336751e-03   7.77515732e-03   7.62205091e-03
   7.29473546e-03   7.11646126e-03   6.89290607e-03   6.64780050e-03
   6.43619228e-03   6.06785733e-03   5.72355481e-03   5.42111779e-03
   5.32572180e-03   5.12593012e-03   4.93751480e-03   4.81152749e-03
   4.70552613e-03   4.48539742e-03   4.37429819e-03   4.29042913e-03
   4.14129885e-03   4.09550383e-03   3.97791365e-03   3.77519751e-03
   3.73569960e-03   3.56489057e-03   3.48715829e-03   3.42085679e-03
   3.39094400e-03   3.20350532e-03   3.11265032e-03   3.09297501e-03
   3.06311250e-03   3.02930118e-03